In [1]:
import argparse
import folium
import pandas as pd
import colourmap as cm

In [2]:
def get_top_clusters_df(df: pd.DataFrame, n: int) -> pd.DataFrame:
    df_most_used_clusters = df.groupby(by=["class"], as_index=False).count().sort_values(by="Адрес", ascending=False).head(n)
    df_most_used_clusters = df_most_used_clusters.loc[df_most_used_clusters["Адрес"] > 100]
    return df.loc[df["class"].isin(df_most_used_clusters["class"])]

In [3]:
def create_marker(x, map):
    coordinates = tuple(x["Геопозиция"][1:-1].split(","))
    marker = folium.CircleMarker(location = coordinates,  
                                 fill_opacity = 0.6, 
                                 radius = x["weight"]*20,
                                 popup = f"{x['weight'], x.name, x['class']}",
                                 # fill_color = ,
                                 color = x["color"]).add_to(map)



In [4]:
def create_df(path, weight_name, top=None):
    df = pd.read_csv(path, index_col=0)
    df = df.drop_duplicates()

    if top is not None:
        df = get_top_clusters_df(df, top)

    max_weight = df[weight_name].max()
    df["weight"] = df[weight_name].apply(lambda loc: loc/max_weight)

    df["color"], palette = cm.fromlist(df["class"], scheme="hex", cmap="Spectral")

    return df

In [5]:
def create_map(df: pd.DataFrame, file_name: str) -> None:
    print("Creating map")
    spb = folium.Map(location=[59.9386, 30.3141], min_lat= 59.5, min_lon=29.5, max_lat=60.5, max_lon=30.8, max_bounds=True, zoom_start=10)
    df.apply(lambda x: create_marker(x, spb), axis=1)
    spb.save(f"../data/html/{file_name}.html")

In [6]:
df_residents = create_df("../data/result/Residents_with_class.csv", weight_name="Кол-во жильцов")
create_map(df_residents, file_name="residents")

Creating map


In [7]:
df_subways = create_df("../data/result/subway_passangers_traffic_with_class.csv", weight_name="Пассажирооборот")
create_map(df_subways, file_name="subways")

Creating map
